NET1

In [1]:
import torch
import torchvision
import torchaudio
import librosa
print("Everything loaded fine ✅")

Everything loaded fine ✅


In [6]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

def get_dataloaders(data_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((180,180)), # Resize to 180x180 pixels
        transforms.ToTensor() # Convert to tensor
    ])

    dataset = datasets.ImageFolder(root=data_dir, transform=transform)

    # Split the dataset into training, validation and test sets (70% train, 20% val, 10% test)
    total_length = len(dataset)
    train_length = int(0.7 * total_length)
    val_length = int(0.2 * total_length)
    test_length = total_length - train_length - val_length
    train_set, val_set, test_set = random_split(dataset, [train_length, val_length, test_length])

    # Create data loaders for each set
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

In [ ]:
data_path = "./Data/images_original"

train_loader, val_loader, test_loader = get_dataloaders(data_path)

for images, labels in train_loader:
    print("Batch of images shape:", images.shape)  # (batch_size, channels, height, width)
    print("Batch of labels shape:", labels.shape)  # (batch_size,)
    break  # Just show the first batch

Batch of images shape: torch.Size([32, 3, 180, 180])
Batch of labels shape: torch.Size([32])


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(3*180*180, 512)  # Input layer
        self.fc2 = nn.Linear(512, 128)  # Hidden layer
        self.output = nn.Linear(128, 10)  # Output layer (10 classes)

    def forward(self, x):
        x = x.view(x.size(0), -1) # Flatten the input
        x = F.relu(self.fc1(x)) # Activation function for the first layer
        x = F.relu(self.fc2(x)) # Activation function for the second layer
        x = self.output(x)
        return x

In [12]:
model = Net1()
optimizer = torch.optim.Adam(Net1().parameters(), lr=0.001) 
loss_fn = nn.CrossEntropyLoss()

In [16]:
def train(model, train_loader, val_loader, optimizer, loss_fn, epochs=50):
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

In [17]:
train (model, train_loader, val_loader, optimizer, loss_fn, epochs=50)

Epoch 1/50, Loss: 50.7350
Epoch 2/50, Loss: 50.7319
Epoch 3/50, Loss: 50.7349
Epoch 4/50, Loss: 50.7339
Epoch 5/50, Loss: 50.7351
Epoch 6/50, Loss: 50.7373
Epoch 7/50, Loss: 50.7325
Epoch 8/50, Loss: 50.7343
Epoch 9/50, Loss: 50.7358
Epoch 10/50, Loss: 50.7310
Epoch 11/50, Loss: 50.7364
Epoch 12/50, Loss: 50.7334
Epoch 13/50, Loss: 50.7325
Epoch 14/50, Loss: 50.7356
Epoch 15/50, Loss: 50.7322
Epoch 16/50, Loss: 50.7368
Epoch 17/50, Loss: 50.7353
Epoch 18/50, Loss: 50.7338
Epoch 19/50, Loss: 50.7383
Epoch 20/50, Loss: 50.7328
Epoch 21/50, Loss: 50.7349
Epoch 22/50, Loss: 50.7363
Epoch 23/50, Loss: 50.7348
Epoch 24/50, Loss: 50.7301
Epoch 25/50, Loss: 50.7400
Epoch 26/50, Loss: 50.7360
Epoch 27/50, Loss: 50.7365
Epoch 28/50, Loss: 50.7344
Epoch 29/50, Loss: 50.7336
Epoch 30/50, Loss: 50.7363
Epoch 31/50, Loss: 50.7343
Epoch 32/50, Loss: 50.7336
Epoch 33/50, Loss: 50.7324
Epoch 34/50, Loss: 50.7341
Epoch 35/50, Loss: 50.7338
Epoch 36/50, Loss: 50.7324
Epoch 37/50, Loss: 50.7353
Epoch 38/5

In [18]:
torch.save(model.state_dict(), "net1_fully_connected.pth")